In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, GRU
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import itertools

In [2]:
data = pd.read_csv('hand_landmarks_dataset.csv')

In [3]:
data['label'].value_counts()

label
Doing other things               38184
Swiping Down                     16650
Thumb Up                         16502
Swiping Left                     16243
Sliding Two Fingers Left         16169
Pulling Hand In                  16169
Zooming Out With Two Fingers     16095
Swiping Up                       16095
Sliding Two Fingers Down         15873
Rolling Hand Forward             15873
Zooming In With Full Hand        15799
Zooming In With Two Fingers      15725
Pushing Two Fingers Away         15725
Pushing Hand Away                15614
Zooming Out With Full Hand       15577
Thumb Down                       15577
Pulling Two Fingers In           15466
Sliding Two Fingers Right        15355
Shaking Hand                     15318
Drumming Fingers                 15244
Stop Sign                        14948
No gesture                       14763
Swiping Right                    14689
Sliding Two Fingers Up           14615
Rolling Hand Backward            13986
Turning Hand Clockw

In [4]:
data.isnull().sum()

label            0
label_id         0
path             0
landmark_0_x     0
landmark_0_y     0
                ..
landmark_19_y    0
landmark_19_z    0
landmark_20_x    0
landmark_20_y    0
landmark_20_z    0
Length: 66, dtype: int64

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 434269 entries, 0 to 434268
Data columns (total 66 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   label          434269 non-null  object 
 1   label_id       434269 non-null  int64  
 2   path           434269 non-null  int64  
 3   landmark_0_x   434269 non-null  float64
 4   landmark_0_y   434269 non-null  float64
 5   landmark_0_z   434269 non-null  float64
 6   landmark_1_x   434269 non-null  float64
 7   landmark_1_y   434269 non-null  float64
 8   landmark_1_z   434269 non-null  float64
 9   landmark_2_x   434269 non-null  float64
 10  landmark_2_y   434269 non-null  float64
 11  landmark_2_z   434269 non-null  float64
 12  landmark_3_x   434269 non-null  float64
 13  landmark_3_y   434269 non-null  float64
 14  landmark_3_z   434269 non-null  float64
 15  landmark_4_x   434269 non-null  float64
 16  landmark_4_y   434269 non-null  float64
 17  landmark_4_z   434269 non-nul

In [6]:
data.groupby('label_id').label.value_counts()

label_id  label                        
0         Doing other things               38184
1         Drumming Fingers                 15244
2         No gesture                       14763
3         Pulling Hand In                  16169
4         Pulling Two Fingers In           15466
5         Pushing Hand Away                15614
6         Pushing Two Fingers Away         15725
7         Rolling Hand Backward            13986
8         Rolling Hand Forward             15873
9         Shaking Hand                     15318
10        Sliding Two Fingers Down         15873
11        Sliding Two Fingers Left         16169
12        Sliding Two Fingers Right        15355
13        Sliding Two Fingers Up           14615
14        Stop Sign                        14948
15        Swiping Down                     16650
16        Swiping Left                     16243
17        Swiping Right                    14689
18        Swiping Up                       16095
19        Thumb Down         

In [7]:
X, y = data.drop(columns=['label','label_id', 'path']), data['label_id']

In [8]:
def super_frame(data, sequence_length = 37, num_features = 63):
    raw_data = data.to_numpy()
    n_sequences = len(raw_data) // sequence_length
    raw_data = raw_data[:n_sequences * sequence_length]
    X_seq = raw_data.reshape((n_sequences, sequence_length, num_features))

    # "Суперкадр" — среднее по последовательности
    X_mean = np.mean(X_seq, axis=1, keepdims=True)  # (n_sequences, 1, 63)

    # Добавляем к последовательности
    X_augmented = np.concatenate([X_seq, X_mean], axis=1)  # (n_sequences, 38, 63)

    X = pd.DataFrame(X_augmented.reshape(38*n_sequences, 63))
    
    return X


In [9]:
prep_X = super_frame(X)

In [10]:
minmax = MinMaxScaler()
X_scaled = minmax.fit_transform(prep_X)

window_size_x = 38
window_size_y = 37
n_samples = len(X_scaled) // window_size_x
n_samples_y = len(y) // window_size_y

y_seq = np.array(y[:n_samples_y * window_size_y]).reshape(n_samples_y, window_size_y)

y_seq = y_seq[:, 0] 

X_scaled = X_scaled.reshape(n_samples, 38, 63)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_seq, train_size=0.8, random_state=13)

In [11]:
y_train = to_categorical(y_train, num_classes=27)
y_test = to_categorical(y_test, num_classes=27)

In [13]:
best_acc = 0
best_model = 0
param_grid = {
    'units': [32, 50, 64],
    'dense_units': [8, 16]
}

# Генерируем все комбинации
all_combinations = list(itertools.product(
    param_grid['units'],
    param_grid['dense_units']
))

# Перебор всех комбинаций
for i, (units, dense_units) in enumerate(all_combinations):
    print(f"\nКомбинация {i+1}/{len(all_combinations)}: "
          f"units={units}, dense_units={dense_units}")
    model = Sequential([
        GRU(units, 
            input_shape=(38, 63), 
            kernel_regularizer=l2(0.01), 
            recurrent_regularizer=l2(0.01),                      
            recurrent_dropout=0.3,),
        Dropout(0.3),
        Dense(dense_units, activation='relu'),
        Dense(27, activation='softmax')  
    ])

    model.compile(optimizer='adam', 
                loss='categorical_crossentropy', 
                metrics=['accuracy'])
    
    early_stop = EarlyStopping(
        monitor='val_loss',     # следим за валидационной ошибкой
        patience=10,            # если не улучшается 10 эпох — стоп
        restore_best_weights=True
    )

    checkpoint = ModelCheckpoint(
        f"model_{i}_best.h5",   # имя сохраняемого файла
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    )

    callbacks = [early_stop, checkpoint]

    history = model.fit(
        X_train, y_train, 
        epochs=100, 
        batch_size=32,
        validation_data=(X_test, y_test),
        callbacks=callbacks      # ← добавляем сюда
    )

    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
    print(f"Тестовая точность: {test_acc}")
    if (test_acc > best_acc):
        best_acc = test_acc
        best_model = i

print(f"best_model: {best_model}, best acc of this model: {best_acc}")



Комбинация 1/6: units=32, dense_units=8


c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
291/294 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0542 - loss: 3.8404
Epoch 1: val_loss improved from inf to 3.27841, saving model to model_0_best.h5


294/294 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.0544 - loss: 3.8366 - val_accuracy: 0.0865 - val_loss: 3.2784
Epoch 2/100
289/294 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0881 - loss: 3.2650
Epoch 2: val_loss improved from 3.27841 to 3.20493, saving model to model_0_best.h5


294/294 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.0881 - loss: 3.2645 - val_accuracy: 0.0788 - val_loss: 3.2049
Epoch 3/100
293/294 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0860 - loss: 3.1915
Epoch 3: val_loss improved from 3.20493 to 3.16679, saving model to model_0_best.h5


...

294/294 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.4452 - loss: 2.0184 - val_accuracy: 0.5183 - val_loss: 1.8462
Epoch 82/100
293/294 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4616 - loss: 1.9712
Epoch 82: val_loss did not improve from 1.84623
294/294 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.4616 - loss: 1.9712 - val_accuracy: 0.4625 - val_loss: 1.9321
Epoch 83/100
288/294 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4567 - loss: 2.0060
Epoch 83: val_loss improved from 1.84623 to 1.84315, saving model to model_5_best.h5


294/294 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.4567 - loss: 2.0056 - val_accuracy: 0.5124 - val_loss: 1.8432
Epoch 84/100
294/294 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4610 - loss: 1.9777
Epoch 84: val_loss did not improve from 1.84315
294/294 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.4610 - loss: 1.9777 - val_accuracy: 0.5256 - val_loss: 1.8480
Epoch 85/100
288/294 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4574 - loss: 1.9802
Epoch 85: val_loss did not improve from 1.84315
294/294 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.4575 - loss: 1.9802 - val_accuracy: 0.4919 - val_loss: 1.8851
Epoch 86/100
288/294 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4616 - loss: 1.9606
Epoch 86: val_loss improved from 1.84315 to 1.82055, saving model to model_5_best.h5


294/294 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.4615 - loss: 1.9608 - val_accuracy: 0.5383 - val_loss: 1.8205
Epoch 87/100
294/294 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4684 - loss: 1.9594
Epoch 87: val_loss did not improve from 1.82055
294/294 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.4684 - loss: 1.9594 - val_accuracy: 0.5217 - val_loss: 1.8369
Epoch 88/100
288/294 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4696 - loss: 1.9454
Epoch 88: val_loss did not improve from 1.82055
294/294 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.4696 - loss: 1.9457 - val_accuracy: 0.5179 - val_loss: 1.8332
Epoch 89/100
288/294 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4576 - loss: 1.9838
Epoch 89: val_loss did not improve from 1.82055
294/294 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.4578 - loss: 1.9834 - val_accuracy: 0.4906 - val_loss: 1.8848
Epoch 90/100
294/294 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4630 - loss: 1.9795
Epoch 90: val_loss did not improve f

294/294 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.4816 - loss: 1.9277 - val_accuracy: 0.5149 - val_loss: 1.7992
Epoch 92/100
288/294 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4644 - loss: 1.9575
Epoch 92: val_loss did not improve from 1.79925
294/294 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.4644 - loss: 1.9575 - val_accuracy: 0.5362 - val_loss: 1.8094
Epoch 93/100
292/294 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4812 - loss: 1.9300
Epoch 93: val_loss did not improve from 1.79925
294/294 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.4812 - loss: 1.9301 - val_accuracy: 0.5281 - val_loss: 1.8290
Epoch 94/100
288/294 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4767 - loss: 1.9412
Epoch 94: val_loss did not improve from 1.79925
294/294 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.4767 - loss: 1.9413 - val_accuracy: 0.5328 - val_loss: 1.8039
Epoch 95/100
288/294 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4704 - loss: 1.9659
Epoch 95: val_loss did not improve f

294/294 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.4673 - loss: 1.9401 - val_accuracy: 0.5451 - val_loss: 1.7819
Epoch 97/100
288/294 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4730 - loss: 1.9300
Epoch 97: val_loss did not improve from 1.78193
294/294 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.4729 - loss: 1.9303 - val_accuracy: 0.5239 - val_loss: 1.8152
Epoch 98/100
288/294 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4681 - loss: 1.9503
Epoch 98: val_loss did not improve from 1.78193
294/294 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.4682 - loss: 1.9502 - val_accuracy: 0.5034 - val_loss: 1.8781
Epoch 99/100
288/294 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4741 - loss: 1.9388
Epoch 99: val_loss did not improve from 1.78193
294/294 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.4741 - loss: 1.9386 - val_accuracy: 0.5336 - val_loss: 1.8228
Epoch 100/100
294/294 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4713 - loss: 1.9483
Epoch 100: val_loss did not improve